In [2]:
import numpy
import keras
from keras.models import Model
from keras.layers import Activation
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import os

Using TensorFlow backend.


In [3]:
root_path = r"C:\Users\vikky\Documents\Third Year\Perk Lab\181101_Spine\181101_Spine"

train_path = os.path.join(root_path, "train")
valid_path = os.path.join(root_path, "valid")
test_path = os.path.join(root_path, "test")

In [4]:
image_size = 32
train_batch_size = 100

In [5]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True,
    batch_size=train_batch_size)

valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True,
    batch_size=train_batch_size)

test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True)

Found 2037 images belonging to 2 classes.
Found 715 images belonging to 2 classes.
Found 747 images belonging to 2 classes.


In [6]:
#Create train and test batches
train_batches.batch_size = train_batches.n
x_train, y_train = next(train_batches)

test_batches.batch_size = test_batches.n
x_test, y_test = next(test_batches)

In [7]:
print("x_test.shape: {}".format(x_test.shape))
print("y_test.shape: {}".format(y_test.shape))

print("x_train.shape: {}".format(x_train.shape))
print("y_train.shape: {}".format(y_train.shape))

x_test.shape: (747, 32, 32, 1)
y_test.shape: (747, 2)
x_train.shape: (2037, 32, 32, 1)
y_train.shape: (2037, 2)


In [8]:
input_img = Input(shape = (32, 32, 1))

In [9]:
#Build the layers
def inception(input_img):
    tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
    tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output

In [10]:
#Create the model
def inceptionModel():

    inputs = Input(shape = (32, 32, 1))

    x = inception(inputs)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(input=inputs, output=predictions)

    model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.001),
                 metrics=['accuracy'])
    return model

In [11]:
modelA = inceptionModel()
modelA.fit(x_train, y_train, epochs=10, shuffle=True,  validation_split=0.1)
score = modelA.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

C:\Users\vikky\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


Train on 1833 samples, validate on 204 samples
Epoch 1/10
1833/1833 [==============================] - 99s 54ms/step - loss: 0.4915 - acc: 0.7883 - val_loss: 0.2409 - val_acc: 0.8971
Epoch 2/10
1833/1833 [==============================] - 111s 61ms/step - loss: 0.1829 - acc: 0.9302 - val_loss: 0.1071 - val_acc: 0.9608
Epoch 3/10
1833/1833 [==============================] - 112s 61ms/step - loss: 0.1053 - acc: 0.9558 - val_loss: 0.0995 - val_acc: 0.9706
Epoch 4/10
1833/1833 [==============================] - 96s 52ms/step - loss: 0.0589 - acc: 0.9804 - val_loss: 0.0645 - val_acc: 0.9706
Epoch 5/10
1833/1833 [==============================] - 98s 53ms/step - loss: 0.0582 - acc: 0.9765 - val_loss: 0.2310 - val_acc: 0.9069
Epoch 6/10
1833/1833 [==============================] - 1185s 646ms/step - loss: 0.0390 - acc: 0.9858 - val_loss: 0.0444 - val_acc: 0.9853
Epoch 7/10
1833/1833 [==============================] - 108s 59ms/step - loss: 0.0388 - acc: 0.9836 - val_loss: 0.0520 - val_acc: 0.